#### TOP 30K

Esta tarea se tiene que ejecutar los martes por la mañana, previo checkeo que los datos están subidos a https://www.ritmogestion.es/listas . Si no están subidos la función dará error. El usuario para entrar en la web es *nmarin* y la contraseña *3-5B0G4W1qWfI6b6*. 

In [1]:
# Se importan las librerias que se van a usar.
import os
import sys
import requests
from sys import argv
import shutil
from utils import get_dates_from_week
from utils import get_main_artist
from utils import get_info_from_track_and_artist
import utils
import importlib
importlib.reload(utils)
import time
from datetime import datetime, timedelta
import pandas as pd 
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException
import write_to_reporting_db as db
import subprocess
from selenium.webdriver.common.by import By

In [2]:
# Variables y rutas de las carpetas que vamos a usar durante el codigo
usuario = os.getlogin()
ruta_script = os.getcwd()
carpeta = os.path.join(os.path.dirname(ruta_script),'1_BBDD','6_0_TOP_30K' ,'Back_up_semanas')
carpeta_no_backup = os.path.join(os.path.dirname(ruta_script),'1_BBDD','6_0_TOP_30K')
print(f'La carpeta es: {carpeta} \n Mientras que la carpeta para el no back_up es: {carpeta_no_backup}')

# Metemos el año y la semana que vamos a ejecutar
year = int(input("Por favor, ingrese un número de AÑO que vamos a añadir: "))
week = int(input("Por favor, ingrese un número de SEMANA que vamos a añadir: "))
# Generamos la variable del trimestre
if week < 14:
    quarter = 1
elif week < 27:
    quarter = 2
elif week < 40: 
    quarter = 3
elif week < 53: 
    quarter = 4
else: print('Semana fuera de rango')
print(f'El trimestre al que pertenece la semana {week} es el {quarter}.')   

# Verificar si el número ingresado coincide con el número de filas del DataFrame
if week > 53 or week < 1:
    print(f"El número ingresado para la semana está mal, tiene que ser un número entre 1 y 52.")
    sys.exit()
print('El número de semana está comprendido entre los valores adecuados.')

if usuario.lower() not in ['alco003', 'nuria01']:
    sys.exit('Error: el valor de la variable usuario: no es válido')
print(f'El usuario: {usuario} es válido.')

print(f'Vas a ejecutar la semana {week}, del año {year}, y el trimestre {quarter}')

# Generamos las fechas que vamos a necesitar
fecha_inicio = get_dates_from_week(week,year)['inicio']
fecha_fin = get_dates_from_week(week,year)['fin']
year_inicio = fecha_inicio[:4] 
mes_inicio = fecha_inicio[5:7]
dia_inicio = fecha_inicio[8:]
year_fin = fecha_fin[:4] 
mes_fin = fecha_fin[5:7]
dia_fin = fecha_fin[8:]
fecha = get_dates_from_week(week,year)['inicio']
print(f'La fecha de inicio es: {fecha_inicio}')
print(f'La fecha de fin es: {fecha_fin}')

La carpeta es: c:\Users\nuria01\Sony Music Entertainment\ESP Business Intelligence - General\1_DATA ANALYTICS\1_BBDD\6_0_TOP_30K\Back_up_semanas 
 Mientras que la carpeta para el no back_up es: c:\Users\nuria01\Sony Music Entertainment\ESP Business Intelligence - General\1_DATA ANALYTICS\1_BBDD\6_0_TOP_30K
El trimestre al que pertenece la semana 34 es el 3.
El número de semana está comprendido entre los valores adecuados.
El usuario: NURIA01 es válido.
Vas a ejecutar la semana 34, del año 2024, y el trimestre 3
La fecha de inicio es: 2024-08-16
La fecha de fin es: 2024-08-22


In [3]:
def obtener_top30k_ritmogestion(year, week):
    username = 'nmarin'
    password = '3-5B0G4W1qWfI6b6'
    nombre_lista = 'Top 30.000 Streaming' 
    base_url = "https://ritmogestion.es"
    auth_endpoint = "/listas/ws/login"
    auth_url = base_url + auth_endpoint
    
    auth_data = {
        "username": username,
        "password": password
    }
    
    auth_headers = {
        "Content-Type": "application/json"
    }
    
    response = requests.post(auth_url, json=auth_data, headers=auth_headers)
    
    if response.status_code == 200:
        token = response.json().get("data", {}).get("token")
        if not token:
            raise Exception("Autenticación exitosa, pero no se encontró ningún token en la respuesta.")
    else:
        raise Exception(f"Error en la autenticación: {response.status_code} - {response.text}")
    
    lista_dict = {
        "Top 100 Full track": "D0",
        "Top 100 Canciones": "D3",
        "Top Radio": "D4",
        "Top Radio Musical": "D5",
        "Top 100 Álbumes Ventas": "V0",
        "Top 20 Recopilaciones": "V1",
        "Top 1.000 Álbumes": "V10",
        "Top 100 Vinilos": "V11",
        "Top DVD Musical": "V2",
        "Top 200 Streaming": "V3",
        "Top Singles": "V4",
        "Top 200 Artistas y Recopilaciones": "V5",
        "Top 100 Streaming Álbumes": "V6",
        "Top 30.000 Streaming": "V7",
        "Top 100 Álbumes": "V8"
    }
    
    list_type = lista_dict.get(nombre_lista)
    if not list_type:
        raise ValueError(f"Error: No se encontró el tipo de lista para {nombre_lista}")
    
    list_endpoint = "/listas/ws/getChart"
    list_url = base_url + list_endpoint
    
    list_headers = {
        "Authorization": f"Bearer {token}"
    }
    
    list_params = {
        "year": year,
        "week": week,
        "type": list_type,
        "lang": "es"
    }
    
    response = requests.get(list_url, headers=list_headers, params=list_params)
    
    if response.status_code == 200:
        data = response.json()
        chart_data = data.get("data", {}).get("chart", [])
        df = pd.DataFrame(chart_data)
        print('Ya tenemos el dataframe creado')
    else:
        raise Exception(f"Error al obtener los datos de la lista: {response.status_code} - {response.text}")
    
    dtypes_completo = {
        '%_dif': str,
        'ad_supported': str,
        'año': int,
        'artista': str,
        'audiencia': int,
        'audiencia_total': int,
        'certificados': str,
        'code': str,
        'comprobar_posible_certificado': str,
        'distribuidor_int': str,
        'fecha_lanzamiento': str,
        'label': str,
        'Lista': str,
        'posicion_maxima': int,
        'premium': str,
        'SEA_acumuladas': int,
        'SEA_semana_actual': int,
        'sello': str,
        'semana': int,
        'semana_actual': int,
        'semana_anterior': str,
        'semanas_lista': int,
        'streams_acumulados': int,
        'streams_semana_actual': int,
        'streams_semana_anterior': int,
        'titulo': str,
        'tocadas': int,
        'total_acumuladas': int,
        'total_semana_actual': int,
        'unidades_acumuladas': int,
        'unidades_acumuladas_adsupported': int,
        'unidades_acumuladas_premium': int,
        'unidades_semana_actual': int,
        'ventas_acumuladas': int,
        'ventas_semana_actual': int,
        'ventas_semana_anterior': int
    }

    print('paso 1')
    
    dtypes_df = {col: dtypes_completo[col] for col in df.columns if col in dtypes_completo}
    
    for col, dtype in dtypes_df.items():
        if dtype in [int, float]:
            print(col)
            df[col] = df[col].fillna(0).astype(dtype)
        else:
            df[col] = df[col].astype(dtype)
    
    print('paso 2')

    
    return df

df = obtener_top30k_ritmogestion(year = year, week = week)
print(df.shape)

Ya tenemos el dataframe creado
paso 1
año
semana
semana_actual
posicion_maxima
streams_semana_actual
streams_semana_anterior
streams_acumulados
unidades_semana_actual
unidades_acumuladas_adsupported
unidades_acumuladas_premium
unidades_acumuladas
paso 2
(30000, 23)


In [5]:
2+2

4

In [4]:
top_10000_folder = carpeta
week_path = os.path.join(top_10000_folder, f'W{week:02}')
#Crear carpeta para la semana
if not os.path.isdir(week_path):
    os.mkdir(week_path)

input_filename = os.path.join(week_path, f'top10000_streaming_w_{week:02}_{year}.csv')
df.to_csv(input_filename, encoding='latin-1', errors='ignore')
print(f'EL ARCHIVO SE ENCUENTRA EN {input_filename}')
output_filename = f'{os.path.splitext(input_filename)[0]}_corregido.csv'

df_output = pd.DataFrame()
df_output["Año"] = df["año"].apply(int)
df_output['Quarter'] = quarter
df_output["Semana"] = df["semana"].apply(int)
list_date = datetime.strptime(fecha, '%Y-%m-%d')
df_output['Fecha'] = pd.to_datetime(list_date, format="%Y-%m-%d")
df_output["Posición"] = df["semana_actual"].apply(int)
df_output["Pos. Anterior"] = df["semana_anterior"].fillna(0).apply(lambda x: int(x) if x != 'None' else 0)
df_output["Pos. Máxima"] = df["posicion_maxima"].apply(int)
df_output["Main Artist"] = '-'
df_output["Artista"] = df["artista"]
df_output["Título"] = df["titulo"]
df_output["Sello"] = df["sello"]
df_output['Label'] = df['label']
df_output['Distribuidor'] = df['distribuidor_int']
df_output["Streams"] = df["streams_semana_actual"]
df_output["Streams Anterior"] = df["streams_semana_anterior"]
df_output["Streams Acum."] = df["streams_acumulados"]
df_output['AdSupported'] = df['ad_supported']
df_output["Premium"] = df["premium"]
df_output['% Dif.'] = df['%_dif']
df_output["Code"] = df["code"]
df['fecha_lanzamiento'] = df['fecha_lanzamiento'].replace('None', pd.NaT)
df_output["Release date"] = pd.to_datetime(df['fecha_lanzamiento'], format="%d-%m-%Y", errors='coerce')

print('Creamos columna "Artista principal" que nos permitirá trabajar con las categorizaciones.') 

# Incluimos '-' en las filas de 'Artista' vacías para eliminar 'Nan' que daba error. 
df_output.loc[df['artista'].isnull(), 'Main Artist'] = '-'
df_output.loc[df_output['Artista'].isna()==True, 'Artista'] = '-'
# df_output['Artista'] = df_output['Artista'].fillna('-')
print('Aplicamos la funcion de utils get_main_artist.')
df_output['Main Artist'] = df_output['Artista'].apply(get_main_artist)
df_output['Main Artist'] = df_output['Main Artist'].apply(get_main_artist)

# NUEVO: HACER ESTO EN LUGAR DE LA REGEX
# quitar las comillas de la columna Main Artist
df_output['Main Artist'] = df_output['Main Artist'].str.replace('"', '')
print('Estos son los artistas que están mal: ')
for idx in df_output.index:
    item = df_output.loc[idx, 'Main Artist']
    if ';' in item:
        # print(item)
        df_output.loc[idx, 'Main Artist'] = item.split(';')[0]
# suponiendo que tu DataFrame se llama pd_30k
df_output['Artista'] = df_output['Artista'].str.replace('"', '')
df_output['Artista'] = df_output['Artista'].str.replace('; ', ' / ')
df_output['Artista'] = df_output['Artista'].str.replace(' ;', ' / ')
df_output['Artista'] = df_output['Artista'].str.replace(';', ' / ')

print('Incluimos - en las columnas de sello, label y distribuidor vacías para eliminar Nan, que daba error.') 
df_output.loc[df_output['Sello'].isnull(), 'Sello'] = '-'
df_output.loc[df_output['Label'].isnull(), 'Label'] = '-'
df_output.loc[df_output['Distribuidor'].isnull(), 'Distribuidor'] = '-'
df_output.loc[df_output['Distribuidor']== 'None', 'Distribuidor'] = '-'

# Creamos lista de 'labels' y 'sellos' con los nombres de distribuidor que están en esas
#  columnas, para facilitar las modificaciones. 
labels = ['ALTAFONTE', 'DISTRO_KID', 'INTERACTIVE', 'DITTO MUSIC', 'LA CUPULA MUSIC', 
        'INGROOV', 'SYMPHONIC', 'THE ORCHARD', 'MERLIN', 'ONERPM', 'LA_CUPULA_MUSIC',
        'LA CÚPULA', 'TUNECORE', 'CDBABY', 'ROUTENOTE', 'DITTO_MUSIC', 'CD BABY', 
        'APARATAJE MUSIC GROUP']
sellos = ['EMPIRE']

# Bucle para incluir el nombre del distribuidor, cuando está contenido en 'Label'. 
for label in labels:
    df_output.loc[df_output['Label'].str.contains(label), 'Distribuidor'] = label

# Bucle para incluir el nombre del distribuidor, cuando está contenido en 'Sello'. 
for sello in sellos:
    df_output.loc[df_output['Sello'].str.contains(sello), 'Distribuidor'] = sello

print('Modificación "manual" de los nombres de distribuidores para que sean siempre iguales.')  
df_output.loc[df_output['Distribuidor'] == 'DISTRO_KID', 'Distribuidor'] = 'DISTRO KID'
df_output.loc[df_output['Distribuidor'] == 'DISTROKID', 'Distribuidor'] = 'DISTRO KID'
df_output.loc[df_output['Distribuidor'] == 'DITTO_MUSIC', 'Distribuidor'] = 'DITTO MUSIC'
df_output.loc[df_output['Distribuidor'] == 'INGROOV', 'Distribuidor'] = 'INGROOVES'
df_output.loc[df_output['Distribuidor'] == 'LA_CUPULA_MUSIC', 'Distribuidor'] = 'LA CUPULA MUSIC'
df_output.loc[df_output['Distribuidor'] == 'LA CÚPULA', 'Distribuidor'] = 'LA CUPULA MUSIC'

df_output['SPNL'] = ''

# Modificacmos manualmente aquellas filas que tengan que ser 'Frontline'
frontline_date = list_date - timedelta(weeks = 52 * 1.5)
old_frontline_date = list_date - timedelta(weeks = 52 * 3.5)
type(frontline_date)

# Creamos nueva columna: 'SPNL' y la llenamos con el valor ''
df_output['SPNL'] = ''
df_output.loc[df_output['Release date'] < old_frontline_date , 'SPNL'] = 'Catálogo'
df_output.loc[df_output['Release date'] >= frontline_date, 'SPNL'] = 'Frontline'
df_output.loc[(df_output['Release date'] >= old_frontline_date) & (df_output['Release date'] < frontline_date), 'SPNL'] = 'Old frontline'

# Asumimos que las canciones que no tienen Release Date son muy antiguas (por inspección) y asignamos catálogo. 
df_output.loc[df_output['Release date'].isnull(), 'SPNL'] = 'Catálogo'

# Creamos nueva columna: 'Novedad' con los posibles valores 'Entrada', 'Novedad' o vacía. 
df_output['Novedad'] = '-'
df_output.loc[df_output['Pos. Anterior'] == 0, 'Novedad'] = 'Entrada'

new_date = list_date - timedelta(days=6)
df_output.loc[(df_output['Pos. Anterior'] == 0) & (df_output['Release date'] >= new_date), 'Novedad'] = 'Novedad'

print('Rellenar ISRC cada semana')

# Cuando la columna CODE empiece por letra, asignaremos a ISRC el mismo valor que el de
#  la columna CODE. 
df_output['ISRC'] = df_output.loc[df_output['Code'].str.contains(r'^[A-Za-z]'), 'Code']
df_output.loc[~df_output['Code'].str.contains(r'^[A-Za-z]'), 'ISRC'] = '-'

# Cuando la columna CODE empiece por número, tenemos que buscar el valor para ISRC. 
# Primero buscaremos en Track_DB y si no lo encontramos, buscaremos en Spotify. 
df_track_db = pd.read_excel(os.path.join(os.path.dirname(ruta_script),'1_BBDD','6_0_TOP_30K','track_db.xlsx'))

df_output['key'] = df_output['Título']+ '_________'+df_output['Main Artist']
df_track_db['key'] = df_track_db['Titulo']+ '_________'+df_track_db['Main Artist']

tracks_without_isrc = df_output.loc[~df_output['Code'].str.contains(r'^[A-Za-z]')]

for key in tracks_without_isrc['key']:
    isrc = df_track_db.loc[df_track_db['key'] == key, 'Spotify ISRC']
    if len(isrc) > 0:
        df_output.loc[df_output['key'] == key, 'ISRC'] = list(isrc)[0]

print('Ecncapsulado en una función para evitar ejecutarlo, obitene el isrc via Spotify de las canciones que no lo tienen tras buscarlo en track_db')
def get_info_from_spotify(df_output):
    tracks_without_isrc = df_output.loc[df_output['ISRC'] == '-']
    tracks = tracks_without_isrc['Título']
    artists = tracks_without_isrc['Main Artist']
    info = [get_info_from_track_and_artist(track, artist)[0] for track, artist in zip(tracks, artists) ]
    df_info = pd.DataFrame(info, columns=['name','release_date','id','isrc','artists'])

fecha_buena = []
for i in range(len(df_output.Fecha)):
    fecha_buena.append(get_dates_from_week(int(df_output['Semana'][i]), int(df_output['Año'][i]), format='%d/%m/%Y')['inicio'])
df_output['Fecha'] = fecha_buena

print(f'Guardamos el top 10000 de la semana {output_filename}') 
df_output.to_csv(output_filename, sep=";", index=False, encoding='latin-1', errors='ignore')

top_10000_folder_no_backup = carpeta_no_backup

acumulado_filename = os.path.join(top_10000_folder_no_backup, 'top_10000_streaming_2021.csv')
acumulado_backup_filename = os.path.join(top_10000_folder,f'W{week:02}', f'top_10000_streaming_2021_backup_{week-1:02}.csv')

print('Hacemos backup del acumulado de la semana anterior')
shutil.copy2(acumulado_filename, acumulado_backup_filename)


EL ARCHIVO SE ENCUENTRA EN c:\Users\nuria01\Sony Music Entertainment\ESP Business Intelligence - General\1_DATA ANALYTICS\1_BBDD\6_0_TOP_30K\Back_up_semanas\W34\top10000_streaming_w_34_2024.csv
Creamos columna "Artista principal" que nos permitirá trabajar con las categorizaciones.
Aplicamos la guncion de utils get_main_artist.
Estos son los artistas que están mal: 
Incluimos - en las columnas de sello, label y distribuidor vacías para eliminar Nan, que daba error.
Modificación "manual" de los nombres de distribuidores para que sean siempre iguales.
Rellenar ISRC cada semana
Ecncapsulado en una función para evitar ejecutarlo, obitene el isrc via Spotify de las canciones que no lo tienen tras buscarlo en track_db
Guardamos el top 10000 de la semana c:\Users\nuria01\Sony Music Entertainment\ESP Business Intelligence - General\1_DATA ANALYTICS\1_BBDD\6_0_TOP_30K\Back_up_semanas\W34\top10000_streaming_w_34_2024_corregido.csv
Hacemos backup del acumulado de la semana anterior


'c:\\Users\\nuria01\\Sony Music Entertainment\\ESP Business Intelligence - General\\1_DATA ANALYTICS\\1_BBDD\\6_0_TOP_30K\\Back_up_semanas\\W34\\top_10000_streaming_2021_backup_33.csv'

In [5]:
# Actualizamos el acumulado con la semana actual añadiendolo al final
df_acum = pd.read_csv(acumulado_filename, sep=";", encoding='latin-1')
df_output['Release date'] = pd.to_datetime(df_output['Release date'], errors='coerce').dt.strftime('%d/%m/%Y')
print(df_output['Release date'].tail(1))

C:\Users\nuria01\AppData\Local\Temp\2/ipykernel_29648/737899958.py:2: DtypeWarning: Columns (14,15,16,17,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acum = pd.read_csv(acumulado_filename, sep=";", encoding='latin-1')


29999    18/12/2021
Name: Release date, dtype: object


In [6]:
df_acum[df_acum.Año == 2024]['Año'].groupby(df_acum.Semana).count()

Semana
1     30000
2     30000
3     30000
4     30000
5     30000
6     30000
7     30000
8     30000
9     30000
10    30000
11    30000
12    30000
13    30000
14    30000
15    30000
16    30000
17    30000
18    30000
19    30000
20    30000
21    30000
22    30000
23    30000
24    30000
25    30000
26    30000
27    30000
28    30000
29    30000
30    30000
31    30000
32    30000
33    30000
Name: Año, dtype: int64

In [7]:
# Cambiamos a mano un tema de anuel aa que es TO y aparece como empire
condicion = (df_output['Main Artist'] == 'ANUEL AA') & (df_output['Título'] == 'MEJOR QUE YO')
condicion_chencho = (df_output['Main Artist'] == 'CHENCHO CORLEONE')
condicion_fuerza_regida = (df_output ['Main Artist'] == 'FUERZA REGIDA')
condicion_seya = (df_output['Main Artist'] == 'GIMS') & (df_output['Título'] == 'SEYA')
condicion_babymonster = (df_output['Main Artist'] == 'BABYMONSTER') & (df_output['Título'] == 'FOREVER')
condicion_ozuna = (df_output['Main Artist'] == 'OZUNA') & (df_output['Título'] == 'RAZONES')

# Actualiza los valores de la columna "distribuidor"
df_output.loc[condicion, 'Distribuidor'] = 'THE ORCHARD'
df_output.loc[condicion_chencho, 'Distribuidor'] = 'THE ORCHARD'
df_output.loc[condicion_fuerza_regida, 'Distribuidor'] = 'THE ORCHARD'
df_output.loc[condicion_seya, 'Distribuidor'] = 'BELIEVE DIGITAL'
df_output.loc[df_output['Artista'] == 'BLESSD & MALUMA', 'Distribuidor'] = 'ADA'
df_output.loc[condicion_babymonster, 'Distribuidor'] = 'OTROS'
df_output.loc[condicion_ozuna, 'Distribuidor'] = 'THE ORCHARD'

# Lista de títulos de canciones que hay que cambiar
titulos = ["THESE WALLS", "END OF AN ERA", "FALLING FOREVER", "WHATCHA DOING", "FRENCH EXIT", "HAPPY FOR YOU", "ANYTHING FOR LOVE"]
df_output.loc[(df_output['Main Artist'] == 'DUA LIPA') & (df_output['Título'].isin(titulos)), 'Distribuidor'] = 'WARNER MUSIC'
print('Punto C')

df_output.to_csv(acumulado_filename, sep=";", index=False, encoding='latin-1',errors='ignore', mode='a', header=False)


Punto C


In [8]:
df_output[df_output.Año == 2024]['Año'].groupby(df_output.Semana).count()

Semana
34    30000
Name: Año, dtype: int64

In [9]:
df_tabla_repDB = pd.read_csv(output_filename, sep=";", encoding='latin-1')
table_repDB='sandbox.nmarin_top10k_spain'
user_repDB='nmarin'
password_repDB='9oGiKRWDkPhF'

db.write_results_into_db_chunked(df_tabla_repDB,table_repDB,user_repDB,password_repDB,chunk=10000)

print('¡¡¡¡PROCESO TERMINADO CON EXITO!!!!')


Total rows 30000

 Chunk: 1 of 3
Successfully wrote into DB
Closed connection to reporting-db.smeanalyticsapps.com

 Chunk: 2 of 3
Successfully wrote into DB
Closed connection to reporting-db.smeanalyticsapps.com

 Chunk: 3 of 3
Successfully wrote into DB
Closed connection to reporting-db.smeanalyticsapps.com
¡¡¡¡PROCESO TERMINADO CON EXITO!!!!


In [10]:
import pandas as pd
import os

ruta_script = os.getcwd()
ruta_top30K = os.path.join(os.path.dirname(ruta_script),'1_BBDD','6_0_TOP_30K')
acumulado_filename = os.path.join(ruta_top30K, 'top_10000_streaming_2021.csv')
dtypes= {'Streams': str}

df_acum = pd.read_csv(acumulado_filename, sep=";", encoding='latin-1', dtype=dtypes)
# df_acum = df_acum.drop(columns=['Semanas_creciendo_Artista'])

for column in dtypes.keys():
        if column in df_acum:
            df_acum[column] = df_acum[column].str.replace('[^\d]', '', regex=True).astype(int)

df_acum['Streams'] = df_acum['Streams'].apply(lambda x: x.replace('.', '') if isinstance(x, str) and '.' in x else x).astype(int)
print('Paso 1')


C:\Users\nuria01\AppData\Local\Temp\2/ipykernel_29648/2249689679.py:9: DtypeWarning: Columns (14,15,16,17,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acum = pd.read_csv(acumulado_filename, sep=";", encoding='latin-1', dtype=dtypes)


Paso 1


In [11]:
df_acum.columns

Index(['Año', 'Quarter', 'Semana', 'Fecha', 'Posición', 'Pos. Anterior',
       'Pos. Máxima', 'Main Artist', 'Artista', 'Título', 'Sello', 'Label',
       'Distribuidor', 'Streams', 'Streams Anterior', 'Streams Acum.',
       'AdSupported', 'Premium', '% Dif.', 'Code', 'Release date', 'SPNL',
       'Novedad', 'Spotify ISRC', 'ISRC', 'key', 'Release date str',
       'Semanas_creciendo', 'Semanas_creciendo_Artista'],
      dtype='object')

In [12]:
# Ordena el DataFrame por 'Main Artist', 'Título' y 'Año' para asegurarte de que los registros estén en orden cronológico.
df = df_acum.sort_values(by=['Main Artist', 'Título', 'Año', 'Semana'])

# Inicializa la columna 'Evolutivo' con ceros.
df['Semanas_creciendo'] = 0

# Itera a través de las filas del DataFrame y calcula el evolutivo.
current_artist = None
current_title = None
current_growth = 0
print('Paso 2')

Paso 2


In [13]:

for index, row in df.iterrows():
    if row['Main Artist'] == current_artist and row['Título'] == current_title:
        # Si es la misma canción y el mismo artista que la semana anterior, verifica si el número de Streams aumentó.
        if row['Streams'] > previous_streams:
            current_growth += 1
        else:
            current_growth = 0
    else:
        # Si es una canción diferente o un artista diferente, reinicia el contador.
        current_artist = row['Main Artist']
        current_title = row['Título']
        current_growth = 0
    
    # Actualiza la columna 'Evolutivo' en el DataFrame.
    df.at[index, 'Semanas_creciendo'] = current_growth
    
    # Almacena los Streams actuales para la próxima comparación.
    previous_streams = row['Streams']

print('Paso 3')

Paso 3


In [14]:
df['Semanas_creciendo'] = df['Semanas_creciendo'].astype(int)
df = df.sort_values(by=['Año', 'Semana', 'Posición'], ascending=[True, True, True])
print('Paso 4')


Paso 4


In [15]:
df.columns

Index(['Año', 'Quarter', 'Semana', 'Fecha', 'Posición', 'Pos. Anterior',
       'Pos. Máxima', 'Main Artist', 'Artista', 'Título', 'Sello', 'Label',
       'Distribuidor', 'Streams', 'Streams Anterior', 'Streams Acum.',
       'AdSupported', 'Premium', '% Dif.', 'Code', 'Release date', 'SPNL',
       'Novedad', 'Spotify ISRC', 'ISRC', 'key', 'Release date str',
       'Semanas_creciendo', 'Semanas_creciendo_Artista'],
      dtype='object')

In [16]:

##### AHORA PARA ARTISTAS 
# Ordena el DataFrame por 'Main Artist', y 'Año' para asegurarte de que los registros estén en orden cronológico.
df = df.drop(columns=['Semanas_creciendo_Artista'])

df_artistas = df.groupby(['Main Artist', 'Año', 'Semana'])['Streams'].sum().reset_index()

# Inicializa la columna 'Evolutivo' con ceros.
df_artistas['Semanas_creciendo_Artista'] = 0

# Itera a través de las filas del DataFrame y calcula el evolutivo.
current_artist = None
current_growth = 0
print('Paso 2')
for index, row in df_artistas.iterrows():
    if row['Main Artist'] == current_artist:
        # Si es la misma canción y el mismo artista que la semana anterior, verifica si el número de Streams aumentó.
        if row['Streams'] > previous_streams:
            current_growth += 1
        else:
            current_growth = 0
    else:
        # Si es una canción diferente o un artista diferente, reinicia el contador.
        current_artist = row['Main Artist']
        current_growth = 0
    
    # Actualiza la columna 'Evolutivo' en el DataFrame.
    df_artistas.at[index, 'Semanas_creciendo_Artista'] = current_growth
    
    # Almacena los Streams actuales para la próxima comparación.
    previous_streams = row['Streams']

print('Paso 3')

df = df.merge(df_artistas[['Main Artist', 'Año', 'Semana', 'Semanas_creciendo_Artista']], on=['Main Artist', 'Año', 'Semana'], how='left')
df['Semanas_creciendo_Artista'] = df['Semanas_creciendo_Artista'].fillna(0)
df['Semanas_creciendo_Artista'] = df['Semanas_creciendo_Artista'].astype(int)
print('Paso 4')

Paso 2
Paso 3
Paso 4


In [17]:
df['Semanas_creciendo'] = df['Semanas_creciendo'].fillna(0)
df['Semanas_creciendo'] = df['Semanas_creciendo'].astype(int)

In [19]:
df.to_csv(acumulado_filename, sep=";", index=False, encoding='latin-1', header=True)
print('Paso 5')

Paso 5


cambios nuria


- cambios en los sellos

In [20]:
import pandas as pd
import re

# Supongamos que tienes un DataFrame llamado 'df'

# Definir una función para eliminar los dígitos que preceden a 'Records DK'
def eliminar_digitos(texto):
    patron = r'\d+\s*RECORDS DK'  # Patron para encontrar dígitos seguidos de 'Records DK'
    return re.sub(patron, 'RECORDS DK', texto)

# Definir una función para quitar los años de los valores de la columna 'Sello'
def quitar_anios(texto):
    patron = r'^\d+\s+'  # Patrón para encontrar el año al principio del texto
    return re.sub(patron, '', texto)

# Aplicar la función a la columna 'Sello'
df['Label'] = df['Label'].apply(quitar_anios)
df['Sello'] = df['Sello'].apply(quitar_anios)
df['Distribuidor'] = df['Distribuidor'].apply(quitar_anios)

# Aplicar la función a la columna 'Sello'
df['Label'] = df['Label'].apply(eliminar_digitos)
df['Sello'] = df['Sello'].apply(eliminar_digitos)
df['Distribuidor'] = df['Distribuidor'].apply(eliminar_digitos)


TypeError: expected string or bytes-like object

In [ ]:
import pandas as pd

# 1. Eliminar todos los "Label" que tengan un formato similar: 1311288 RECORDS DK.
df['Label'] = df['Label'].apply(lambda x: '-' if 'RECORDS DK' in x else x)
df['Sello'] = df['Sello'].apply(lambda x: '-' if 'RECORDS DK' in x else x)
df['Distribuidor'] = df['Distribuidor'].apply(lambda x: '-' if 'RECORDS DK' in x else x)

# 2. y 3. Modificar sello y distribuidor según las condiciones dadas.
df['Sello'] = df['Sello'].apply(lambda x: 'Distrokid' if 'PK INTERACTIVE / DISTROKID' in x else 'BOA MUSIC SPAIN' if 'ALTAFFONTE (BOA MUSIC SPAIN)' in x else x)
df['Distribuidor'] = df['Distribuidor'].apply(lambda x: 'PK INTERACTIVE' if 'PK INTERACTIVE / DISTROKID' in x else 'ALTAFONTE' if 'ALTAFFONTE (BOA MUSIC SPAIN)' in x else x)

# 4. Corregir ANGELS&#039; DAWN RECORDS.
df['Distribuidor'] = df['Distribuidor'].str.replace("ANGELS&#039;", "ANGELS'")

# 5. Quitar el final de APARATAJE MUSIC GROUP,LLC.
df['Label'] = df['Label'].str.replace(" APARATAJE MUSIC GROUP,LLC", " APARATAJE MUSIC GROUP")

# 6. Reemplazar interrogantes con '-'.
df.replace('???????-??????', '-', inplace=True)

# 7. Reemplazar "N.A." con '-'.
df.replace('N.A.', '-', inplace=True)

# 8. Unificar CANZION GROUP.
df['Label'] = df['Label'].str.replace("CANZION GROUP LP", "CANZION GROUP")

# 9. Separar CDBABY como CD BABY.
df['Label'] = df['Label'].str.replace("CDBABY", "CD BABY")

# 10. Cambiar CARINCO AG a CARINCO.
df.replace('CARINCO AG', 'CARINCO', inplace=True)

# 11. Cambiar DISTRO KID a DISTROKID.
df['Label'] = df['Label'].str.replace("DISTRO KID", "DISTROKID")

# 12. Unificar diferentes formas de Entertainment One.
df['Label'] = df['Label'].apply(lambda x: 'ENTERTAINMENT ONE' if 'ENTERTAINMENT ONE' in x else x)
df['Sello'] = df['Sello'].apply(lambda x: 'ENTERTAINMENT ONE' if 'ENTERTAINMENT ONE' in x else x)
df['Distribuidor'] = df['Distribuidor'].apply(lambda x: 'ENTERTAINMENT ONE' if 'ENTERTAINMENT ONE' in x else x)

# 13. Corregir KEYVIEM MUSIC GROUP/EQUITY DISTRIBUTIO.
df['Distribuidor'] = df['Distribuidor'].str.replace("KEYVIEM MUSIC GROUP / EQUITY DISTRIBUTIO", "KEYVIEM MUSIC GROUP/EQUITY DISTRIBUTION")

# 14. Reducir EMPIRE DISTRIBUTION a EMPIRE.
df.replace('EMPIRE DISTRIBUTION', 'EMPIRE', inplace=True)


In [ ]:

# 15. Llenar valores de Sello con Label si está vacío o '-'.
df['Sello'].fillna(df['Label'], inplace=True)
df.loc[df['Sello'] == '-', 'Sello'] = df.loc[df['Sello'] == '-', 'Label']
# 16. Llenar valores de Label con Sello si está vacío o '-'.
df['Label'].fillna(df['Sello'], inplace=True)
df.loc[df['Label'] == '-', 'Label'] = df.loc[df['Label'] == '-', 'Sello']

# 17. Llenar valores de Label con Distribuidor si está vacío o '-'.
df['Label'].fillna(df['Distribuidor'], inplace=True)
df.loc[df['Label'] == '-', 'Label'] = df.loc[df['Label'] == '-', 'Distribuidor']

# 18. Llenar valores de Distribuidor con Label si está vacío o '-'.
df['Distribuidor'].fillna(df['Label'], inplace=True)
df.loc[df['Distribuidor'] == '-', 'Distribuidor'] = df.loc[df['Distribuidor'] == '-', 'Label']

# 19. Reemplazar valores en Distribuidor según las condiciones dadas.
df.replace({'Distribuidor': {'INTERACTIVE': 'PK INTERACTIVE', 'IGROOVE AG': 'IGROOVE MUSIC', 'LA CUPULA': 'LA CUPULA MUSIC'}}, inplace=True)

# 20. Reemplazar &amp; con &
df.replace('&amp;', '&', regex=True, inplace=True)

# 21. Cambiar KRAKEN DISTRIBUCION MUSICAL SL a KRAKEN DISTRIBUTION
df.replace('KRAKEN DISTRIBUCION MUSICAL SL', 'KRAKEN DISTRIBUTION', inplace=True)

# 22. Cambiar L 10 MUSIC a L10 MUSIC
df['Label'] = df['Label'].str.replace('L 10 MUSIC', 'L10 MUSIC')
df['Sello'] = df['Sello'].str.replace('L 10 MUSIC', 'L10 MUSIC')
df['Distribuidor'] = df['Distribuidor'].str.replace('L 10 MUSIC', 'L10 MUSIC')

# 23. Cambiar [PIAS] y algo más a PIAS DIGITAL
df['Label'] = df['Label'].apply(lambda x: 'PIAS DIGITAL' if '[PIAS]' in x else x)
df['Sello'] = df['Sello'].apply(lambda x: 'PIAS DIGITAL' if '[PIAS]' in x else x)
df['Distribuidor'] = df['Distribuidor'].apply(lambda x: 'PIAS DIGITAL' if '[PIAS]' in x else x)

# 24. Quitar ",", ";" o "." al final de los valores en Label, Sello y Distribuidor
df['Label'] = df['Label'].str.rstrip(',;.')
df['Sello'] = df['Sello'].str.rstrip(',;.')
df['Distribuidor'] = df['Distribuidor'].str.rstrip(',;.')


In [ ]:
df.to_csv(acumulado_filename, sep=";", index=False, encoding='latin-1', header=True)

In [ ]:
import pandas as pd

# Supongamos que tienes un DataFrame llamado 'df' con las columnas: Label, Sello, Distribuidor, Streams, y otras columnas si las hay.

# Agrupar por las columnas especificadas y sumar los Streams
df_agrupado = df.groupby(['Label', 'Sello', 'Distribuidor'])['Streams'].sum().reset_index()
print('hola')
# Ordenar el DataFrame de mayor a menor por Streams
df_agrupado = df_agrupado.sort_values(by='Streams', ascending=False)
print('hola')
# Resetear el índice y eliminar la columna de índice anterior
df_agrupado = df_agrupado.reset_index(drop=True).drop_duplicates()
print('hola')
print(df_agrupado.shape)
# Guardar el DataFrame en un archivo Excel
df_agrupado.to_excel('resultado_agrupado.xlsx', index=False)

hola
hola
hola
(20490, 4)


- Cambios sellos

In [ ]:
import pandas as pd
import os

ruta_script = os.getcwd()
ruta_top30K = os.path.join(os.path.dirname(ruta_script),'1_BBDD','6_0_TOP_30K')
acumulado_filename = os.path.join(ruta_top30K, 'top_10000_streaming_2021.csv')
ruta_sellos_nuevos = os.path.join(ruta_top30K, 'Limpieza_sellos.xlsx')
dtypes= {'Streams': str}

df_acum = pd.read_csv(acumulado_filename, sep=";", encoding='latin-1', dtype=dtypes)
df_sellos = pd.read_excel(ruta_sellos_nuevos)
for column in dtypes.keys():
        if column in df_acum:
            df_acum[column] = df_acum[column].str.replace('[^\d]', '', regex=True).astype(int)

df_acum['Streams'] = df_acum['Streams'].apply(lambda x: x.replace('.', '') if isinstance(x, str) and '.' in x else x).astype(int)

df_acum['Fecha'] = pd.to_datetime(df_acum['Fecha'], format='%d/%m/%Y')

filtered_df = df_acum[df_acum['Fecha'] >= '2023-04-01']

# Hacer un merge de df_acum con df_sellos basado en Label y Sello_origen
df_acum = pd.merge(df_acum, df_sellos, how='left', left_on='Label', right_on='Sello_origen')

# Crear la nueva columna Sello_nuevo en df_acum
df_acum['Sello_nuevo'] = df_acum['Sello_nuevo'].fillna(df_acum['Label'])

# Eliminar las columnas innecesarias después del merge
df_acum = df_acum.drop(columns=['Sello_origen'])

# Eliminar los espacios y comas finales de la columna Sello_nuevo
df_acum['Sello_nuevo'] = df_acum['Sello_nuevo'].str.rstrip(' ,')

C:\Users\alco003\AppData\Local\Temp\1\ipykernel_16232\542470447.py:10: DtypeWarning: Columns (14,15,16,17,25,26) have mixed types. Specify dtype option on import or set low_memory=False.
  df_acum = pd.read_csv(acumulado_filename, sep=";", encoding='latin-1', dtype=dtypes)


Index(['Año', 'Quarter', 'Semana', 'Fecha', 'Posición', 'Pos. Anterior',
       'Pos. Máxima', 'Main Artist', 'Artista', 'Título', 'Sello', 'Label',
       'Distribuidor', 'Streams', 'Streams Anterior', 'Streams Acum.',
       'AdSupported', 'Premium', '% Dif.', 'Code', 'Release date', 'SPNL',
       'Novedad', 'Spotify ISRC', 'ISRC', 'key', 'Release date str',
       'Semanas_creciendo', 'Semanas_creciendo_Artista'],
      dtype='object')

In [ ]:
df_acum.to_csv(acumulado_filename, sep=";", index=False, encoding='latin-1', header=True)